In [1]:
from pathlib import Path
import pickle
import warnings

import pyterrier as pt
import pandas as pd

from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

from config import DATASETS, MODELS, AVAILABLE_DATASETS_PER_MODEL


warnings.filterwarnings('ignore')

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
RUN_PREFIX = 'ows_ltr'
RUN_DIR = Path('../runs/')

In [3]:
def create_run(model_name: str, run_suffix: str):
    dataset = pt.get_dataset(f'irds:ir-benchmarks/{DATASETS[run_suffix]}')

    extract_features = MODELS[model_name](tira, dataset)

    bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

    with open(f'models/lmart_{model_name}.pkl', 'rb') as f:
        lmart = pickle.load(f)

    pipeline = (bm25 % 1000) >> extract_features >> pt.ltr.apply_learned_model(lmart, form="ltr")

    run = pipeline(dataset.get_topics(variant='query'))

    run_name = f'{RUN_PREFIX}_{model_name}'
    run_file = RUN_DIR / run_name / f'{run_name}.{run_suffix}'
    run_file.parent.mkdir(exist_ok=True, parents=True)

    pt.io.write_results(run, run_file, format='trec', run_name=run_name)

In [4]:
for model_name, available_datasets in AVAILABLE_DATASETS_PER_MODEL.items():
    for run_suffix in available_datasets:
        print(f'Creating {RUN_PREFIX}_{model_name}.{run_suffix}...')
        create_run(model_name, run_suffix)

Creating ows_ltr_wows_only.train_2023...
Creating ows_ltr_wows_only.WT...
Creating ows_ltr_wows_only.ST...
Creating ows_ltr_wows_only.LT...
Creating ows_ltr_wows_only.train_2024...
Creating ows_ltr_wows_only.lag6...
Creating ows_ltr_wows_only.lag8...
Creating ows_ltr_wows_base_rerank.train_2023...
Creating ows_ltr_wows_base_rerank.WT...
Creating ows_ltr_wows_base_rerank.ST...
Creating ows_ltr_wows_base_rerank.LT...
Creating ows_ltr_wows_base_rerank.train_2024...
Creating ows_ltr_wows_base_rerank.lag6...
Creating ows_ltr_wows_base_rerank.lag8...
Creating ows_ltr_wows_all_rerank.train_2023...
Creating ows_ltr_wows_all_rerank.WT...
Creating ows_ltr_wows_all_rerank.train_2024...
Creating ows_ltr_wows_all_rerank.lag6...
Creating ows_ltr_wows_all_rerank.lag8...
Creating ows_ltr_wows_rerank_and_keyquery.train_2024...
Creating ows_ltr_wows_rerank_and_keyquery.lag6...
Creating ows_ltr_wows_rerank_and_keyquery.lag8...
Creating ows_ltr_wows_rerank_and_reverted_index.train_2024...
Creating ows_ltr